In [49]:

vcf_file_address =  "22_5smp_4k.vcf" #"a/22_2k.vcf" #  "22_all.vcf"  "a/22_2k.vcf" #  "a/22_2k.vcf"#   #"22_allps_200.vcf"  
vcf_file = open(vcf_file_address,'r');


blockid_blocks_s1 = []
allele_blocks_s1 = []
varpos_blocks_s1 = []


blockid_blocks_s2 = []
allele_blocks_s2 = []
varpos_blocks_s2 = []

blockid_blocks_s3 = []
allele_blocks_s3 = []
varpos_blocks_s3 = []

blockid_blocks_s4 = []
allele_blocks_s4 = []
varpos_blocks_s4 = []

var_blockids1_blocks_s2 = []   # for each var in s2, shows the s1 blockid
var_blockids1_blocks_s3 = []   # for each var in s3, shows the s1 blockid
var_blockids1_blocks_s4 = []   # for each var in s4, shows the s1 blockid


# for parental that it's only one block per chr so it doesn't have blockid
alleles_s5 = []
varposs_s5 = []


varposs_all = []

num_weird_occurs=0

header_lines_list = []
for line in vcf_file:
    line_strip = line.strip()
    
    if line_strip.startswith('#'):
        header_lines_list.append(line_strip)
        sample_names = line_strip.split('\t')[9:]   # last line of header contains sample name
        #print(sample_names)
        
    else:
        line_parts = line_strip.split('\t')
        
        #gt_flags, s1, sss, ss, s2, s5 = line_parts[8:]
        gt_flags, s1, s2,s3,s4, s5 = line_parts[8:]       # hifi-whatshp illumina-whatshp hifi-hapcut illumina-hapcut true

        varpos = int(line_parts[1])                 # variant position
        
        varposs_all.append(varpos)
        gt_flags_split = gt_flags.split(":")
        
        if "PS" in gt_flags_split:
            idx_ps = gt_flags.split(":").index("PS")
            idx_gt = gt_flags.split(":").index("GT")
            

                                                 ## s5- parental - last sample in the vcf file
            split_s5 = s5.split(":")
            allele_s5 = split_s5[idx_gt]
            blockid_s5 = split_s5[idx_ps]
            
            if allele_s5 != "./.":
                alleles_s5.append(int(allele_s5[0]))
                varposs_s5.append(varpos)

            
            
                                                ###  sample 1 ###
            split_s1 = s1.split(":")
            blockid_s1 = split_s1[idx_ps]
            allele_s1 = split_s1[idx_gt]
            

            if blockid_s1 == ".":                                 # unphased variants for sample one
                pass
            
            elif blockid_s1 in blockid_blocks_s1:
                assert allele_s1[1] == "|", "allele_s1[1] should be | and so phased. A non-phased variant is in the phase block"
                
                #if blockid_s1 == previous_blockid_s1: previous_blockid_s1_idx has been defined in previous variant which is the last idx
                if blockid_s1 != previous_blockid_s1:  
                    # weird situation happens for whatshap in which two variants are in the same block but in between there is/are other phaseblock(s)
                    # note that the unphased variants are ignored sooner and are not considered here
                    previous_blockid_s1_idx = blockid_blocks_s1.index(blockid_s1)
                    #print("two variants are in the same block, but far from each other",varpos)                
                    num_weird_occurs+=1

                allele_blocks_s1[previous_blockid_s1_idx].append( int(allele_s1[0]) )
                varpos_blocks_s1[previous_blockid_s1_idx].append( varpos ) 

                
            else:                                                  # A new block started, it can be the first block
                
                allele_blocks_s1.append( [int(allele_s1[0])] )     # alleles of a phase block (all vars are hetrozygous) 
                varpos_blocks_s1.append( [varpos] )                # variant posistion of a block
                blockid_blocks_s1.append( blockid_s1 )             # add the blockid of the first var  to the list of blockids
                
                previous_blockid_s1 = blockid_s1
                previous_blockid_s1_idx = len(blockid_blocks_s1)-1  # the index of the last block id
                


                
       
    
                                                ###  sample 2 ###
            split_s2 = s2.split(":")
            blockid_s2 = split_s2[idx_ps]
            allele_s2 = split_s2[idx_gt]
            if blockid_s2 == ".":
                pass
            elif blockid_s2 in blockid_blocks_s2:
                assert allele_s2[1] == "|", "allele_s2[1] should be | and so phased. A non-phased variant is in the phase block"
                
                if blockid_s2 != previous_blockid_s2:  
                    previous_blockid_s2_idx = blockid_blocks_s2.index(blockid_s2)
                    num_weird_occurs+=1
                allele_blocks_s2[previous_blockid_s2_idx].append( int(allele_s2[0]) )
                varpos_blocks_s2[previous_blockid_s2_idx].append( varpos ) 
                var_blockids1_blocks_s2[previous_blockid_s2_idx].append( blockid_s1 )
                
            else:                                                 # A new block started   (it can be the first block)
                allele_blocks_s2.append( [int(allele_s2[0])] )     # alleles of a phase block (all vars are hetrozygous) 
                varpos_blocks_s2.append( [varpos] )                # variant posistion of a block
                blockid_blocks_s2.append( blockid_s2 )             # add the blockid of the first var  to the list of blockids
                var_blockids1_blocks_s2.append( [blockid_s1] )  
                previous_blockid_s2 = blockid_s2
                previous_blockid_s2_idx = len(blockid_blocks_s2)-1   # the index of the last block id
                

                                                ###  sample 3 ###
            split_s3 = s3.split(":")
            blockid_s3 = split_s3[idx_ps]
            allele_s3 = split_s3[idx_gt]
            if blockid_s3 == ".":
                pass
            elif blockid_s3 in blockid_blocks_s3:
                assert allele_s3[1] == "|", "allele_s3[1] should be | and so phased. A non-phased variant is in the phase block"
                
                if blockid_s3 != previous_blockid_s3:  
                    previous_blockid_s3_idx = blockid_blocks_s3.index(blockid_s3)
                    num_weird_occurs+=1
                allele_blocks_s3[previous_blockid_s3_idx].append( int(allele_s3[0]) )
                varpos_blocks_s3[previous_blockid_s3_idx].append( varpos ) 
                var_blockids1_blocks_s3[previous_blockid_s3_idx].append( blockid_s1 )
                
            else:                                                 # A new block started   (it can be the first block)
                allele_blocks_s3.append( [int(allele_s3[0])] )     # alleles of a phase block (all vars are hetrozygous) 
                varpos_blocks_s3.append( [varpos] )                # variant posistion of a block
                blockid_blocks_s3.append( blockid_s3 )             # add the blockid of the first var  to the list of blockids
                var_blockids1_blocks_s3.append( [blockid_s1] )  
                previous_blockid_s3 = blockid_s3
                previous_blockid_s3_idx = len(blockid_blocks_s3)-1   # the index of the last block id
                
   
    
                                                ###  sample 3 ###
            split_s4 = s4.split(":")
            blockid_s4 = split_s4[idx_ps]
            allele_s4 = split_s4[idx_gt]
            if blockid_s4 == ".":
                pass
            elif blockid_s4 in blockid_blocks_s4:
                assert allele_s4[1] == "|", "allele_s4[1] should be | and so phased. A non-phased variant is in the phase block"
                
                if blockid_s4 != previous_blockid_s4:  
                    previous_blockid_s4_idx = blockid_blocks_s4.index(blockid_s4)
                    num_weird_occurs+=1
                allele_blocks_s4[previous_blockid_s4_idx].append( int(allele_s4[0]) )
                varpos_blocks_s4[previous_blockid_s4_idx].append( varpos ) 
                var_blockids1_blocks_s4[previous_blockid_s4_idx].append( blockid_s1 )
                
            else:                                                 # A new block started   (it can be the first block)
                allele_blocks_s4.append( [int(allele_s4[0])] )     # alleles of a phase block (all vars are hetrozygous) 
                varpos_blocks_s4.append( [varpos] )                # variant posistion of a block
                blockid_blocks_s4.append( blockid_s4 )             # add the blockid of the first var  to the list of blockids
                var_blockids1_blocks_s4.append( [blockid_s1] )  
                previous_blockid_s4 = blockid_s4
                previous_blockid_s4_idx = len(blockid_blocks_s4)-1   # the index of the last block id
                




print("Num phaseblocks in sample one is ",len(allele_blocks_s1))
print("Num phaseblocks in sample two is ",len(allele_blocks_s2))
print("Num phaseblocks in sample three is ",len(allele_blocks_s3))
print("Num phaseblocks in sample four is ",len(allele_blocks_s4))
print("Num vars in sample five-parental is ",len(varposs_s5))
print("Num all vars is ",len(varposs_all))


print("number of occurance of weird=two variants are in the same block, but far from each other ",num_weird_occurs)
 

Num phaseblocks in sample one is  5
Num phaseblocks in sample two is  5
Num phaseblocks in sample three is  265
Num phaseblocks in sample four is  267
Num vars in sample five-parental is  1306
Num all vars is  3792
number of occurance of weird=two variants are in the same block, but far from each other  0


In [23]:
# for test code
# cd /Volumes/work/myjupyter/phaseyou
#grep "46014176" 22_all.vcf

#blockid_blocks_s2.index("46014176")
#print(allele_blocks_s2[5568],"\n",varpos_blocks_s2[5568])


In [52]:
#%pwd
num_phased_s1=0
for varpos_block_s1 in varpos_blocks_s1:
    num_phased_s1+=len(varpos_block_s1)
    if len(varpos_block_s1)<2:
        print("small",varpos_block_s1)
print(num_phased_s1)


# num_phased_s2=0
# for allele_block_s2 in allele_blocks_s2:
#     num_phased_s2+=len(allele_block_s2)
#     if len(allele_block_s2)<2:
#         print("small,s2",len(allele_block_s2))
# print(num_phased_s2)

num_phased_a=0
for a in var_blockids1_blocks_s2:
    num_phased_a+=len(a)
print(num_phased_a)   
#len(var_blockids1_blocks_s2[100])
#print(var_blockids1_blocks_s2[0])

2446
2581


In [53]:
intersected_blocks_idins1_s2=[] # for each block in s2, the corresponding s1 block id

for var_blockids1_blocks_s2_i in var_blockids1_blocks_s2:
    var_blockids1_blocks_s2_i_set=set(var_blockids1_blocks_s2_i)
    
    if '.' in var_blockids1_blocks_s2_i_set:
        var_blockids1_blocks_s2_i_set.discard('.')
        
    if len(var_blockids1_blocks_s2_i_set)==1:
        intersected_idins1_s2=list(var_blockids1_blocks_s2_i_set)[0]
    if len(var_blockids1_blocks_s2_i_set)==0:
        intersected_idins1_s2=0
    if len(var_blockids1_blocks_s2_i_set)>1:
        # find the most frequent blockid in the phaseblock
        intersected_idins1_s2=max(var_blockids1_blocks_s2_i_set, key=var_blockids1_blocks_s2_i.count) 
    
        
    intersected_blocks_idins1_s2.append(intersected_idins1_s2)


In [55]:
intersected_blocks_idins1_s3=[] # for each block in s3, the corresponding s1 block id

for var_blockids1_blocks_s3_i in var_blockids1_blocks_s3:
    var_blockids1_blocks_s3_i_set=set(var_blockids1_blocks_s3_i)
    
    if '.' in var_blockids1_blocks_s3_i_set:
        var_blockids1_blocks_s3_i_set.discard('.')
        
    if len(var_blockids1_blocks_s3_i_set)==1:
        intersected_idins1_s3=list(var_blockids1_blocks_s3_i_set)[0]
    if len(var_blockids1_blocks_s3_i_set)==0:
        intersected_idins1_s3=0
    if len(var_blockids1_blocks_s3_i_set)>1:
        # find the most frequent blockid in the phaseblock
        intersected_idins1_s3=max(var_blockids1_blocks_s3_i_set, key=var_blockids1_blocks_s3_i.count) 
    
        
    intersected_blocks_idins1_s3.append(intersected_idins1_s3)


In [54]:
intersected_blocks_idins1_s4=[] # for each block in s4, the corresponding s1 block id

for var_blockids1_blocks_s4_i in var_blockids1_blocks_s4:
    var_blockids1_blocks_s4_i_set=set(var_blockids1_blocks_s4_i)
    
    if '.' in var_blockids1_blocks_s4_i_set:
        var_blockids1_blocks_s4_i_set.discard('.')
        
    if len(var_blockids1_blocks_s4_i_set)==1:
        intersected_idins1_s4=list(var_blockids1_blocks_s4_i_set)[0]
    if len(var_blockids1_blocks_s4_i_set)==0:
        intersected_idins1_s4=0
    if len(var_blockids1_blocks_s4_i_set)>1:
        # find the most frequent blockid in the phaseblock
        intersected_idins1_s4=max(var_blockids1_blocks_s4_i_set, key=var_blockids1_blocks_s4_i.count) 
    
        
    intersected_blocks_idins1_s4.append(intersected_idins1_s4)


In [28]:
#print(blockid_blocks_s1)
#for i in range(len(allele_blocks_s1)):
#    print(len(allele_blocks_s1[i]))
    
#print(blockid_blocks_s2)
#for i in range(len(allele_blocks_s2)):
#    print(len(allele_blocks_s2[i]))

In [29]:
# re-ordering  the phase block allel based on parent

number_phaseblocks_s1= len(allele_blocks_s1)

#allele_blocks_s1_new=list(allele_blocks_s1)
for block_i in range(number_phaseblocks_s1): # 
   
    allele_block_s1=allele_blocks_s1[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]


    similiar_count=0
    diss_count=0
    for var_i, varpos in enumerate(varpos_block_s1):
        allele_s1= allele_block_s1[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
            if allele_s1 == allele_s5:
                similiar_count+=1
            else: 
                diss_count+=1
                
    #print(similiar_count, diss_count)          
    if similiar_count < diss_count:
        allele_block_s1_reordered = [1-i for i in allele_block_s1] # converting 0->1 and 1->0
        allele_blocks_s1[block_i]=allele_block_s1_reordered
        #print("it needs to re-order")
    
#         allele_block_s1_new = [1-i for i in allele_block_s1]
#         allele_blocks_s1_new[block_i]=allele_block_s1_new
        


12 7
16 23
it needs to re-order
39 35
5 24
it needs to re-order


In [61]:
number_phaseblocks_s2= len(allele_blocks_s2)

for block_i in range(number_phaseblocks_s2): # 
   
    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]

    intersected_idins1_s2=intersected_blocks_idins1_s2[block_i]
    if intersected_idins1_s2 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s2)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s2)
    similiar_count=0
    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
        if allele_s1==allele_s2: similiar_count+=1
    #print(similiar_count/len(intersect_varpos))
    if similiar_count < len(intersect_varpos)/2:
        #print("need", similiar_count)
        allele_block_s2_reordered= [1-i for i in allele_block_s2]
        allele_blocks_s2[block_i]=allele_block_s2_reordered
        #print("it needs to re-order")



In [63]:
number_phaseblocks_s3= len(allele_blocks_s3)

for block_i in range(number_phaseblocks_s3): # 
   
    allele_block_s3=allele_blocks_s3[block_i]
    varpos_block_s3=varpos_blocks_s3[block_i]

    intersected_idins1_s3=intersected_blocks_idins1_s3[block_i]
    if intersected_idins1_s3 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s3)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s3)
    similiar_count=0
    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s3= allele_block_s3[varpos_block_s3.index(varpos)]
        if allele_s1==allele_s3: similiar_count+=1
    #print(similiar_count/len(intersect_varpos))
    if similiar_count < len(intersect_varpos)/2:
        #print("need", similiar_count)
        allele_block_s3_reordered= [1-i for i in allele_block_s3]
        allele_blocks_s3[block_i]=allele_block_s3_reordered
        #print("it needs to re-order")


        
number_phaseblocks_s4= len(allele_blocks_s4)

for block_i in range(number_phaseblocks_s4): # 
   
    allele_block_s4=allele_blocks_s4[block_i]
    varpos_block_s4=varpos_blocks_s4[block_i]

    intersected_idins1_s4=intersected_blocks_idins1_s4[block_i]
    if intersected_idins1_s4 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s4)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s4)
    similiar_count=0
    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s4= allele_block_s4[varpos_block_s4.index(varpos)]
        if allele_s1==allele_s4: similiar_count+=1
    #print(similiar_count/len(intersect_varpos))
    if similiar_count < len(intersect_varpos)/2:
        #print("need", similiar_count)
        allele_block_s4_reordered= [1-i for i in allele_block_s4]
        allele_blocks_s4[block_i]=allele_block_s4_reordered
        #print("it needs to re-order")



In [65]:
matrix_row_lists=[]
for varpos in varposs_all:
    # for increasing the speed we can use the information that the phase blocks are in order.
    # but not for those weird phase block


    allele_s1='.'
    blockid_s1='.'
    for block_i_s1 in range(len(blockid_blocks_s1)):
        allele_block_s1 = allele_blocks_s1[block_i_s1]
        varpos_block_s1 = varpos_blocks_s1[block_i_s1]

        if varpos in varpos_block_s1:
            allele_s1 = allele_block_s1[varpos_block_s1.index(varpos)]
            blockid_s1 = blockid_blocks_s1[block_i_s1]
            break # when var is found, no need to continue for loop

            
    allele_s2='.'
    blockid_s2='.'
    for block_i_s2 in range(len(blockid_blocks_s2)):
        allele_block_s2 = allele_blocks_s2[block_i_s2]
        varpos_block_s2 = varpos_blocks_s2[block_i_s2]

        if varpos in varpos_block_s2:
            allele_s2 = allele_block_s2[varpos_block_s2.index(varpos)]
            blockid_s2 = blockid_blocks_s2[block_i_s2]
            break # when var is found, no need to continue for loop

    allele_s3='.'
    blockid_s3='.'
    for block_i_s3 in range(len(blockid_blocks_s3)):
        allele_block_s3 = allele_blocks_s3[block_i_s3]
        varpos_block_s3 = varpos_blocks_s3[block_i_s3]

        if varpos in varpos_block_s3:
            allele_s3 = allele_block_s3[varpos_block_s3.index(varpos)]
            blockid_s3 = blockid_blocks_s3[block_i_s3]
            break # when var is found, no need to continue for loop

    allele_s4='.'
    blockid_s4='.'
    for block_i_s4 in range(len(blockid_blocks_s4)):
        allele_block_s4 = allele_blocks_s4[block_i_s4]
        varpos_block_s4 = varpos_blocks_s4[block_i_s4]

        if varpos in varpos_block_s4:
            allele_s4 = allele_block_s4[varpos_block_s4.index(varpos)]
            blockid_s4 = blockid_blocks_s4[block_i_s4]
            break # when var is found, no need to continue for loop

    
            
    if varpos in varposs_s5:
        allele_s5 = alleles_s5[varposs_s5.index(varpos)]
    else: allele_s5= '.'

    if not (allele_s1=='.' and allele_s2=='.' and allele_s3=='.' and allele_s4=='.' and allele_s5=='.'): 
        matrix_row_list=[str(varpos),str(allele_s1)+":"+str(blockid_s1),
                         str(allele_s2)+":"+str(blockid_s2),str(allele_s3)+":"+str(blockid_s3),
                         str(allele_s4)+":"+str(blockid_s4), str(allele_s5)] 
        matrix_row_lists.append(matrix_row_list)

        
# one option can be for loop over blockid_blocks_s1 instead of varposs_all, then search in  blockid_blocks_s1 also for printing sort them to overcome weird phase block
# the issue will be for those variant that are not phased in blockid_blocks_s1 but in blockid_blocks_s2


In [67]:
matrix_address=vcf_file_address+"_matrix5.txt"

matrix_file=open(matrix_address,'w');
matrix_file.write("\t\t".join(["Pos","HiFiWsp","IllmWsp","HiFiHpct","IllmHpct","parental"])+"\n") # "Pos","ONT","CLR","HiFi","Illm","parental"
for matrix_row_list in matrix_row_lists:

        matrix_file.write("\t".join(matrix_row_list)+"\n")

matrix_file.close()
   

In [69]:

diff_list=[]

for block_i in range(len(allele_blocks_s1)): # 
   
    allele_block_s1=allele_blocks_s1[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]


    for var_i, varpos in enumerate(varpos_block_s1):
        allele_s1= allele_block_s1[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
        
            if allele_s1!=allele_s5:
                
                for block_i in range(number_phaseblocks_s2): # 
   
                    allele_block_s2=allele_blocks_s2[block_i]
                    varpos_block_s2=varpos_blocks_s2[block_i]
                    if varpos in varpos_block_s2:
                        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
                        diff_list.append([varpos,allele_s1,allele_s2,allele_s5])
                        #print([varpos,allele_s1,allele_s2,allele_s5],"\n")
                        break

                
                
                
                
                



In [70]:

diff_list_s2=[]

for block_i in range(len(allele_blocks_s2)): # 
   
    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]


    for var_i, varpos in enumerate(varpos_block_s2):
        allele_s2= allele_block_s2[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
        
            if allele_s2!=allele_s5:
                
                for block_i in range(number_phaseblocks_s1): # 
   
                    allele_block_s1=allele_blocks_s1[block_i]
                    varpos_block_s1=varpos_blocks_s1[block_i]
                    if varpos in varpos_block_s1:
                        allele_s1= allele_block_s1[varpos_block_s2.index(varpos)]
                        diff_list_s2.append([varpos,allele_s1,allele_s2,allele_s5])
                        #print([varpos,allele_s1,allele_s2,allele_s5],"\n")
                        break

            

IndexError: list index out of range

In [39]:
hifiok_illno=[]
hifino_illno=[]
hifino_illok=[]

for [varpos, hifi,illu,parnt] in diff_list_s2:
    if illu!=parnt and hifi==parnt:
        hifiok_illno.append(varpos)
    if illu!=parnt and hifi!=parnt:
        #print([varpos, hifi,illu,parnt] )
        hifino_illno.append(varpos)       
    if illu==parnt and hifi!=parnt:
        #print([varpos, hifi,illu,parnt] )
        hifino_illok.append(varpos) 
    
print(len(diff_list_s2),len(hifiok_illno),len(hifino_illok),len(hifino_illno))

56 23 0 33


In [40]:
hifiok_illno=[]
hifino_illno=[]
hifino_illok=[]

for [varpos, hifi,illu,parnt] in diff_list:
    
    if illu!=parnt and hifi==parnt:
        hifiok_illno.append(varpos)
    if illu!=parnt and hifi!=parnt:
        #print([varpos, hifi,illu,parnt] )
        hifino_illno.append(varpos)       
    if illu==parnt and hifi!=parnt:
        #print([varpos, hifi,illu,parnt] )
        hifino_illok.append(varpos) 
    
print(len(diff_list_s2),len(hifiok_illno),len(hifino_illok),len(hifino_illno))

56 0 22 33


In [41]:
allele_s5==allele_s2
[allele_s1,allele_s5,allele_s2]

[1, 1, 1]

In [42]:
similiar_count=0
dissimiliar_count=0
dissimiliar_count_s1ok=0
dissimiliar_count_s2ok=0
dissimiliar_count_exist5=0
similiar_count_s1s2ok=0
similiar_count_s1s2no=0
similiar_count_exist5=0

for block_i in range(number_phaseblocks_s2): # 
   
    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]

    intersected_idins1_s2=intersected_blocks_idins1_s2[block_i]
    if intersected_idins1_s2 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s2)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s2)

    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
        if allele_s1==allele_s2: 
            similiar_count+=1
            if varpos in varposs_s5:
                allele_s5= alleles_s5[varposs_s5.index(varpos)]
                similiar_count_exist5+=1
                if allele_s1==allele_s5 and allele_s2==allele_s5:
                    similiar_count_s1s2ok+=1
                if allele_s1!=allele_s5 and allele_s2!=allele_s5:
                    similiar_count_s1s2no+=1
        else:
            #print(varpos)
            dissimiliar_count+=1
            if varpos in varposs_s5:
                allele_s5= alleles_s5[varposs_s5.index(varpos)]
                dissimiliar_count_exist5+=1
                if allele_s1==allele_s5 and allele_s2!=allele_s5:
                    dissimiliar_count_s1ok+=1
                if allele_s1!=allele_s5 and allele_s2==allele_s5:
                    dissimiliar_count_s2ok+=1
                    

            
#     if similiar_count>1 and dissimiliar_count>1:
#         print(similiar_count)

print(similiar_count,dissimiliar_count,dissimiliar_count_exist5,dissimiliar_count_s1ok,dissimiliar_count_s2ok)
print(similiar_count_exist5,similiar_count_s1s2ok,similiar_count_s1s2no)

830 192 45 23 22
99 66 33


In [43]:
len(allele_block_s2)

11

In [44]:

diff_list=[]
count_simi=0
block_i=1
print("blockid ",blockid_blocks_s1[block_i])

allele_block_s1=allele_blocks_s1[block_i]
varpos_block_s1=varpos_blocks_s1[block_i]

print("Pos","HiFi","Illm","parental","\n")
for var_i, varpos in enumerate(varpos_block_s1):
    allele_s1= allele_block_s1[var_i]

    if varpos in varposs_s5:
        allele_s5= alleles_s5[varposs_s5.index(varpos)]

        if allele_s1!=allele_s5:

            for block_i in range(number_phaseblocks_s2): # 

                allele_block_s2=allele_blocks_s2[block_i]
                varpos_block_s2=varpos_blocks_s2[block_i]
                if varpos in varpos_block_s2:
                    allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
                    diff_list.append([varpos,allele_s1,allele_s2,allele_s5])
                    print(varpos,allele_s1,allele_s2,allele_s5,"\n")
                    break
        if allele_s1==allele_s5:
            count_simi+=1
            


                
print(count_simi)

blockid  16099924
Pos HiFi Illm parental 

16101684 1 1 0 

16104385 1 1 0 

16105717 1 0 0 

16113547 0 0 1 

16117006 0 1 1 

16120074 1 0 0 

16123812 1 0 0 

16137152 1 0 0 

16142002 1 1 0 

16143370 1 1 0 

16143995 1 1 0 

16144768 1 1 0 

16144952 1 1 0 

23


In [45]:
sum1=0
for  a in varpos_blocks_s1:
    sum1+=len(a)
print(sum1)


1087


In [46]:
sum2=0
for  a in varpos_blocks_s2:
    sum2+=len(a)

In [47]:
sum2

1083